### Linkroot WD SWERIK
See https://github.com/swerik-project/riksdagen-persons/issues/27#issuecomment-2456456383

check SWERIKS linkroot - [P12192](https://www.wikidata.org/wiki/Property:P12192)

version 0.1 has progressbar

In [1]:
from datetime import datetime
start_time = datetime.now()
print("Last run: ", datetime.now())

Last run:  2024-11-12 10:09:02.380817


In [2]:
import sys
!{sys.executable} -m pip install wikibaseintegrator


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: /Library/Frameworks/Python.framework/Versions/3.12/bin/python3.12 -m pip install --upgrade pip


In [3]:
from wikibaseintegrator.wbi_helpers import execute_sparql_query
from wikibaseintegrator import WikibaseIntegrator 
from wikibaseintegrator.wbi_config import config as wbi_config

In [4]:
wbi_config['USER_AGENT'] = 'WikibaseIntegrator in PAWS by salgo60'
wbi = WikibaseIntegrator()
results = execute_sparql_query("""
SELECT ?wd ?swerik WHERE {
  ?wd wdt:P12192 ?swerik.
} 
""")


In [5]:
bindings = results["results"]["bindings"]
print(f"Found {len(bindings)} results")
count = 1
#global NrnotValid 
NrnotValid = 0
#global NrValid
NrValid = 0

Found 6177 results


In [6]:
import requests

def checkurl(wd, swerik):
    global NrValid
    global NrnotValid
    base_url = f"https://swerik-project.github.io/person-catalog/{swerik}"
    try:
        response = requests.get(base_url)
        if response.status_code == 200:
            NrValid += 1
        else:
            NrnotValid += 1
            print(f"WD {wd} - {base_url}")
    except requests.exceptions.RequestException as e:
        NrnotValid += 1
        print(f"WD {wd} - {base_url}")
        print(f"Error: {e}")
        return False, f"WD {wd} - {base_url} - Error: {e}"

In [7]:
# pip install tqdm
from tqdm.notebook import tqdm 
from time import sleep
from tqdm import tqdm

In [ ]:
# List to store errors
errors = []
for result in tqdm(bindings, 
                         total=len(bindings), 
                         desc="Processing records"):
    #print (result)

    swerik = result["swerik"]["value"]
    wdurl = result["wd"]["value"]
    wd = str(wdurl).replace("http://www.wikidata.org/entity/","")
    try:
        success, error_message = checkurl(wd, swerik)
        if not success and error_message:
            errors.append(error_message)
    except Exception as e:
        # Store the error details in the list
        errors.append((wd, swerik, str(e)))



Processing records:  12%|██▎                 | 713/6177 [03:05<28:31,  3.19it/s]

WD Q4934552 - https://swerik-project.github.io/person-catalog/i-PCZrYEHwPaEeNTZphEsWTv


Processing records:  15%|██▉                 | 905/6177 [04:05<28:28,  3.09it/s]

WD Q4957371 - https://swerik-project.github.io/person-catalog/i-31gPpUoSm7zqzQckVmfPGy


Processing records:  16%|███▏                | 995/6177 [04:33<27:36,  3.13it/s]

...

Processing records:  42%|████████           | 2612/6177 [13:06<15:19,  3.88it/s]

WD Q98937434 - https://swerik-project.github.io/person-catalog/i-EzcxskgMAVbnq8hM2F2km9
WD Q98937482 - https://swerik-project.github.io/person-catalog/i-HYFwSCrwnemwyJTLMcyqvN


Processing records:  59%|███████████▏       | 3625/6177 [18:31<11:41,  3.64it/s]

WD Q117223085 - https://swerik-project.github.io/person-catalog/i-EQM2NLR1fbN9izUQhjTRGR


Processing records:  60%|███████████▍       | 3710/6177 [18:58<13:09,  3.13it/s]

In [ ]:
# Print the results
print(f"Number of valid URLs: {NrValid}")
print(f"Number of invalid URLs: {NrnotValid}")

if errors:
    print("\nErrors encountered:")
    for wd, swerik, error_msg in errors:
        print(f"Error with wd: {wd}, swerik: {swerik} - {error_msg}")
else:
    print("\nAll records processed without errors.")

In [ ]:
print("End run: ", datetime.now())
print('Time elapsed (hh:mm:ss.ms) {}'.format(datetime.now() - start_time))